<a href="https://colab.research.google.com/github/Sanchit112/FakeNews/blob/master/features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [30]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
real = pd.read_csv("/content/realtweetsfinal.csv")
fake = pd.read_csv("/content/faketweetsfinal.csv")
userreal = pd.read_csv("/content/userreal.csv")
userfake = pd.read_csv("/content/userfake.csv")

In [12]:
real.columns

Index(['date', 'search', 'username', 'tweet', 'nlikes', 'nretweets',
       'nreplies'],
      dtype='object')

In [16]:
userreal.columns

Index(['name', 'username', 'bio', 'location', 'url', 'join_date', 'join_time',
       'tweets', 'following', 'followers', 'likes', 'media', 'private',
       'verified'],
      dtype='object')

In [0]:
userreal = userreal.drop(['id','profile_image_url', 'background_image'],axis=1)
userfake = userfake.drop(['id','profile_image_url', 'background_image'],axis=1)

In [0]:
fakefull = pd.merge(left=real, right=userreal, left_on='username', right_on='username')
realfull = pd.merge(left=fake, right=userfake, left_on='username', right_on='username')

In [18]:
fakefull.columns

Index(['date', 'search', 'username', 'tweet', 'nlikes', 'nretweets',
       'nreplies', 'name', 'bio', 'location', 'url', 'join_date', 'join_time',
       'tweets', 'following', 'followers', 'likes', 'media', 'private',
       'verified'],
      dtype='object')

In [0]:
def features(temp):
    length = len(temp)
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,temp)
    temp = re.sub(r"[^a-zA-Z0-9]+", ' ', temp)
    special = length - len(temp)
    tokens = word_tokenize(temp)
    words = len(tokens)
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    porter = PorterStemmer()
    stems = []
    for t in tokens:    
        stems.append(porter.stem(t))
    useful_words = len(stems)
    return length, len(url), special, words, stems, useful_words

In [0]:
 x = realfull['tweet'].apply(features).to_list()
 y = fakefull['tweet'].apply(features).to_list()

In [0]:
realFeatures = pd.DataFrame(x, columns =['Characters', 'urls', 'SecialCharacters', 'Words', 'Stems', 'UsefulWords']) 
fakeFeatures = pd.DataFrame(y, columns =['Characters', 'urls', 'SecialCharacters', 'Words', 'Stems', 'UsefulWords']) 

In [0]:
fake_tweets = pd.concat([realfull, realFeatures], axis=1, sort=False)
real_tweets = pd.concat([fakefull, fakeFeatures], axis=1, sort=False)

In [0]:
fake_tweets.to_csv("fake_tweets.csv")
real_tweets.to_csv("real_tweets.csv")